In [1]:
import numpy as np
import tensorflow as tf
import random
import matplotlib.pyplot as plt
plt.style.use('ggplot')

seed = 50
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

## LSTM example: Machine translation

### Data

In [2]:
from num2words import num2words

def create_french_to_english_data():
    start = 1
    end = 1000
    
    # Create the dataset
    source = []
    target = []
    for i in range(start, end+1):
        # Convert the number to words in French
        words = '<start> ' + num2words(i, lang='fr') + ' <end>'
        source.append(words)
        words = '<start> ' + num2words(i, lang='en') + ' <end>'
        target.append(words)
    
    return source, target

In [3]:
source, target = create_french_to_english_data()

In [4]:
from sklearn.model_selection import train_test_split

def split_data(source, target):

    train_source, test_source, train_target, test_target = train_test_split(
        source, target, test_size=0.2, random_state=42
    )

    return train_source, test_source, train_target, test_target

In [5]:
train_source, test_source, train_target, test_target = split_data(source, target)

### Prepare sequences

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(oov_token="<OOV>", filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(train_source + train_target)

In [7]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

def prepare_sequences(source, target, tokenizer):
    """
    Prepare the input and output sequences for training the model.

    Parameters:
    dataset (list): The dataset to be processed.
    tokenizer (Tokenizer): The tokenizer to be used.

    Returns:
    tuple: The prepared input and output sequences.
    """


    X_encoder = tokenizer.texts_to_sequences(source)

    Y_encoded = tokenizer.texts_to_sequences(target)
    Y_decoder_input = Y_encoded
    Y_decoder_output = [encoded[1:] + [0] for encoded in Y_encoded]
    
    X_encoder = pad_sequences(X_encoder, maxlen=8, padding='post', truncating='post', dtype='float32')
    Y_decoder_input = pad_sequences(Y_decoder_input, maxlen=8, padding='post', truncating='post', dtype='float32')
    Y_decoder_output = pad_sequences(Y_decoder_output, maxlen=8, padding='post', truncating='post', dtype='float32')

    return X_encoder, Y_decoder_input, Y_decoder_output

In [8]:
X_train_encoder, Y_train_decoder_input, Y_train_decoder_output = prepare_sequences(train_source, train_target, tokenizer)
X_test_encoder, Y_test_decoder_input, Y_test_decoder_output = prepare_sequences(test_source, test_target, tokenizer)

In [9]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, CategoryEncoding, TimeDistributed
from tensorflow.keras.models import Model

def create_lstm_encoder_decoder_model(encoder_vocab_size, decoder_vocab_size, embedding_dim, seq_length):

    # Encoder
    encoder_inputs = Input(shape=(seq_length, encoder_vocab_size), name='encoder_inputs')
    #encoder_embedding = TimeDistributed(CategoryEncoding(num_tokens=encoder_vocab_size, output_mode = "one_hot"))(encoder_inputs)
    encoder_lstm, state_h, state_c = LSTM(units=lstm_units, return_state=True, name='encoder_lstm')(encoder_inputs)
    encoder_states = [state_h, state_c]
    
    # Decoder
    decoder_inputs = Input(shape=(seq_length, decoder_vocab_size), name='decoder_inputs')
    #decoder_embedding = TimeDistributed(CategoryEncoding(num_tokens=encoder_vocab_size, output_mode = "one_hot"))(decoder_inputs)
    decoder_lstm, _, _ = LSTM(
        units=lstm_units, return_sequences=True, return_state=True, name="decoder_lstm"
    )(decoder_inputs, initial_state=encoder_states)
    decoder_dense = Dense(units=decoder_vocab_size, activation='softmax', name='decoder_dense')
    decoder_outputs = decoder_dense(decoder_lstm)
    
    # Model
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Display the model summary
    model.summary(line_length=120)

    return model

In [10]:
# Assuming vocabulary sizes are known
encoder_vocab_size = 59  # Change this to your actual encoder vocabulary size
decoder_vocab_size = 59  # Change this to your actual decoder vocabulary size
embedding_dim = 256  # Dimension of the embedding vectors
lstm_units = 512  # Number of LSTM units
seq_length = 8  # Sequence length for both encoder and decoder

model = create_lstm_encoder_decoder_model(encoder_vocab_size, decoder_vocab_size, embedding_dim, seq_length)

2024-06-12 20:53:01.519810: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Max
2024-06-12 20:53:01.519832: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2024-06-12 20:53:01.519838: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
2024-06-12 20:53:01.519862: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-06-12 20:53:01.519875: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
________________________________________________________________________________________________________________________
 Layer (type)                       Output Shape                        Param #     Connected to                        
 encoder_inputs (InputLayer)        [(None, 8, 59)]                     0           []                                  
                                                                                                                        
 decoder_inputs (InputLayer)        [(None, 8, 59)]                     0           []                                  
                                                                                                                        
 encoder_lstm (LSTM)                [(None, 512),                       1171456     ['encoder_inputs[0][0]']            
                                     (None, 512),                                                                       
                 

In [11]:
X_train_encoder = to_categorical(X_train_encoder, num_classes=len(tokenizer.word_index)+1)
Y_train_decoder_input = to_categorical(Y_train_decoder_input, num_classes=len(tokenizer.word_index)+1)
Y_train_decoder_output = to_categorical(Y_train_decoder_output, num_classes=len(tokenizer.word_index)+1)

In [12]:
Y_train_decoder_output.shape

(800, 8, 59)

In [13]:
from tensorflow.keras.callbacks import EarlyStopping

callback_es = EarlyStopping(monitor='val_loss', patience=20)
history = model.fit(
    [X_train_encoder, Y_train_decoder_input],
    Y_train_decoder_output,
    batch_size=64,
    epochs=500,
    validation_split=0.2,
    callbacks=[callback_es]
)

Epoch 1/500


2024-06-12 20:53:02.960943: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 2s 70ms/step - loss: 3.3823 - accuracy: 0.2936 - val_loss: 2.3479 - val_accuracy: 0.4430
Epoch 2/500
10/10 [==============================] - 0s 16ms/step - loss: 2.2554 - accuracy: 0.4922 - val_loss: 1.9528 - val_accuracy: 0.5336
Epoch 3/500
10/10 [==============================] - 0s 18ms/step - loss: 1.8290 - accuracy: 0.5461 - val_loss: 1.7343 - val_accuracy: 0.6211
Epoch 4/500
10/10 [==============================] - 0s 18ms/step - loss: 1.6365 - accuracy: 0.5934 - val_loss: 1.5272 - val_accuracy: 0.5859
Epoch 5/500
10/10 [==============================] - 0s 16ms/step - loss: 1.4191 - accuracy: 0.6137 - val_loss: 1.2502 - val_accuracy: 0.6477
Epoch 6/500
10/10 [==============================] - 0s 17ms/step - loss: 1.1335 - accuracy: 0.6604 - val_loss: 1.0429 - val_accuracy: 0.6719
Epoch 7/500
10/10 [==============================] - 0s 18ms/step - loss: 0.9899 - accuracy: 0.6764 - val_loss: 0.9537 - val_accuracy: 0.6812
Epoch 8/500
10/10 

In [14]:
def translate(text):
    encoder_input = tokenizer.texts_to_sequences([text]) 
    encoder_input = pad_sequences(encoder_input, maxlen=8, padding='post', truncating='post', dtype='float32')
    encoder_input = to_categorical(encoder_input, num_classes=len(tokenizer.word_index)+1)
    
    T_y = 8
    n_y = len(tokenizer.word_index)+1
    decoder_input = np.zeros([1, T_y, n_y])
    word = '<start>'
    idx = tokenizer.word_index[word]
    
    translated_text = ''
    for t in range(T_y):
        translated_text += word + ' '
        decoder_input[0, t, idx] = 1
        y_hat = model.predict([encoder_input, decoder_input])
        idx = np.argmax(y_hat[0, t, :])
        word = tokenizer.index_word[idx]
        if word == '<end>':
            translated_text += word + ' '
            break

    return translated_text

In [16]:
def generate_example_translations():
    indices = np.random.choice(200, 5, replace=False)
    for i in range(5):    
        text = test_source[indices[i]]
        translated_text = translate(text)
        print ("----------------")
        print ("French text:", test_source[indices[i]])
        print ("Translation:", translated_text)
        print ("Ground truth:", test_target[indices[i]])

    return None

generate_example_translations()

1/1 [==============================] - 0s 10ms/step
----------------
French text: <start> neuf cent quatre-vingt-dix-neuf <end>
Translation: <start> nine hundred and ninety nine <end> 
Ground truth: <start> nine hundred and ninety-nine <end>
1/1 [==============================] - 0s 10ms/step
----------------
French text: <start> vingt-six <end>
Translation: <start> twenty six <end> 
Ground truth: <start> twenty-six <end>
1/1 [==============================] - 0s 11ms/step
----------------
French text: <start> cinquante-six <end>
Translation: <start> fifty six <end> 
Ground truth: <start> fifty-six <end>
1/1 [==============================] - 0s 11ms/step
----------------
French text: <start> cent huit <end>
Translation: <start> one hundred and eight <end> 
Ground truth: <start> one hundred and eight <end>
1/1 [==============================] - 0s 11ms/step
----------------
French text: <start> trois cent soixante et onze <end>
Translation: <start> three hundred and seventy one <end> 
